### Import Required Libraries

In [ ]:
import os

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
pd.options.display.float_format = '{:.5f}'.format

### Create Helper to Choose Data Based On Index

In [ ]:
def select_dates(df, start, end):
    mask = (df.index >= start) & (df.index <= end)
    return df[mask]

### Read Input

In [ ]:
df = pd.read_csv("data/users.csv")
df.head()

### Change Index to Date

In [ ]:
df['time'] = pd.to_datetime(df.date, format="%Y%m%d")
df.index = df.time
df = df.drop(['time', 'date'], axis=1)
df.head()

### Select Train and Test Set

In [ ]:
df_train = select_dates(df, start="2017-01-01", end="2017-10-30")
df_test = select_dates(df, start="2017-11-01", end="2017-12-31")

### Plot Time-Series

In [ ]:
ts = df_train['users']
ts.plot(figsize=(15, 6))

### Decomposition Plot

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 11, 9
decomposition1 = sm.tsa.seasonal_decompose(ts, model='additive') #multiplicative # additive
fig1 = decomposition1.plot()
plt.show()

### Check Stationarity

In [ ]:
def test_stationarity(timeseries, window=7):    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=window,center=False).mean()
    rolstd = timeseries.rolling(window=window,center=False).std()
    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)    
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(ts)

### Make Series Stationary

In [ ]:
ts_log = np.log(ts)
ts_log_diff = ts_log - ts_log.shift()
ts_log_diff.dropna(inplace=True)

In [ ]:
test_stationarity(ts_log_diff)

### Draw ACF-PACF Plot

In [ ]:
def draw_acf_pacf(ts, nlags):
    lag_acf = acf(ts, nlags=nlags)
    lag_pacf = pacf(ts, nlags=nlags, method='ols')
    #Plot ACF: 
    plt.subplot(121) 
    plt.plot(lag_acf)
    plt.axhline(y=0,linestyle='--',color='gray')
    plt.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
    plt.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
    plt.title('Autocorrelation Function')
    #Plot PACF:
    plt.subplot(122)
    plt.plot(lag_pacf)
    plt.axhline(y=0,linestyle='--',color='gray')
    plt.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
    plt.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
    plt.title('Partial Autocorrelation Function')
    plt.tight_layout()

In [ ]:
draw_acf_pacf(ts_log_diff, nlags=7)

### Fit Model

In [ ]:
model = ARIMA(ts_log, order=(7, 1, 3))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))

### Testing

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_log = pd.Series(ts_log.ix[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(ts.values)
plt.plot(predictions_ARIMA.values)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts)))

### Predictions

In [ ]:
start_pred = len(results_ARIMA.fittedvalues) + 1
duration = ((df_test.index[-1])-(ts.index[-1])).days
s1 = results_ARIMA.fittedvalues
predictions_ARIMA_diff = s1.append(results_ARIMA.predict(start=start_pred, end=start_pred + duration))
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_log = pd.Series(ts_log.ix[0], index=predictions_ARIMA_diff_cumsum.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(df_test, color='red')
plt.plot(predictions_ARIMA[start_pred-1:start_pred+duration], color='blue')
plt.title('RMSE: %.4f'% np.sqrt(np.sum((predictions_ARIMA[start_pred-1:start_pred+duration].values-df_test.values)**2)/len(df_test.values)))

### Rolling Forecast Example